In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/transformers/2b/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b/2/gemma-2b.gguf
/kaggle/input/gemma/transformers/2b/2/config.json
/kaggle/input/gemma/transformers/2b/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b/2/tokenizer.json
/kaggle/input/gemma/transformers/2b/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b/2/.gitattributes
/kaggle/input/gemma/transformers/2b/2/tokenizer.model
/kaggle/input/gemma/transformers/2b/2/generation_config.json
/kaggle/input/santa-2024/sample_submission.csv


In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
import random, pickle, math, warnings
#warnings.simplefilter('ignore')

p = '/kaggle/input/santa-2024/sample_submission.csv'
df = pd.read_csv(p) # 	id 	text
print(df['text'].map(lambda x: len(str(x).split(' '))).values)

[ 10  20  20  30  50 100]


In [3]:
tokens = df.text.values
tokens = ' '.join(tokens)
tokens = tokens.split(' ')
tokens = dict(Counter(tokens))
print(len(tokens))
#print(tokens)
print('-'*20)
print(' '.join(k for k in tokens))

89
--------------------
advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh workshop stocking holly jingle beard naughty nice sing of is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy wonder believe dream hope peace joy merry season greeting card wrapping paper bow cookie milk star wish wreath angel to in that have it not with as you from we kaggle


In [4]:
import transformers, torch
import gc, os, logging
from math import exp
from typing import List, Optional, Union

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
DEVICE = torch.device('cpu')
MODEL_PATH = "/kaggle/input/gemma/transformers/2b/2"

class ParticipantVisibleError(Exception):
    print(Exception)
    pass

class PerplexityCalculator:
    def __init__(self):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            MODEL_PATH, device_map="auto", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def get_perplexity(self, input_texts: Union[str, List[str]]) -> Union[float, List[float]]:
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts
        loss_list = []

        with torch.no_grad():
            for text in input_texts:
                text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
                minputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False)
                minputs = {k: v.to(self.device) for k, v in minputs.items()}
                output = self.model(**minputs, use_cache=False)
                logits = output['logits']
                slogits = logits[..., :-1, :].contiguous()
                slabels = minputs['input_ids'][..., 1:].contiguous()
                loss = self.loss_fct(slogits.view(-1, slogits.size(-1)), slabels.view(-1))
                sequence_loss = loss.sum() / len(loss)
                loss_list.append(sequence_loss.cpu().item())

        ppl = [exp(i) for i in loss_list]
        return ppl[0] if single_input else ppl

scorer = PerplexityCalculator()

<class 'Exception'>


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokens = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh workshop stocking holly jingle beard naughty nice sing of is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy wonder believe dream hope peace joy merry season greeting card wrapping paper bow cookie milk star wish wreath angel to in that have it not with as you from we kaggle"
text_with_special = f"{scorer.tokenizer.bos_token}{tokens}{scorer.tokenizer.eos_token}"
minputs = scorer.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False,)
minputs['input_ids']

tensor([[     2, 104828,  67905,  52931,   2730,  43485, 136507,   7727, 165493,
          29138, 103360,   1513,  80108,    541,   5376,   2734,   9902,   6109,
          44528,    573,   6284,   3354,  10084,    578,    597,   1731,  23675,
          42768,  17196,  22867,  12083, 138763, 198447,  16621,  99946,  16573,
           2660,  14111, 155702,  20257,  77515, 108548, 204063,  38175,  97840,
           4866,   2800,    576,    603,   7812,   3532,  10228,    748,  14660,
           1965, 215898,  28162,  83096,    881,   9437,   8529, 112671, 149218,
           7815,   1312,    869,   9471,  23144,  13171,  25720,  24754,   2398,
           7474,  12849,   5144,   4564,   6523,   4077,   7124,  10300,  46301,
           3891,  32338,   4076,  56178,   4368,   7181,  17467,   9512,   2343,
           6199,  58409,  22448,    577,    575,    674,    791,    665,    780,
            675,    685,    692,    774,    783, 124555,   2315,      1]])

In [6]:
past = {'reindeer mistletoe elf scrooge gingerbread chimney fireplace ornament family advent': 495.6812574407127,
 'reindeer mistletoe elf gingerbread ornament advent family fireplace chimney sleep drive walk jump laugh give and bake the night scrooge': 514.3236249412977,
 'magi yuletide cheer grinch carol holly jingle naughty nice polar beard sleigh chimney workshop nutcracker holiday ornament decorations gifts stocking': 327.18895045897995,
 'sleigh of the magi yuletide cheer grinch is unwrap gifts decorations ornament holly stocking and chimney naughty nice polar beard nutcracker visit workshop eat relax carol sing holiday cheer jingle': 327.33220436011106,
 'wreath merry have and season hohoho to you from the star of wonder workshop that it not with joy we believe hope peace fruitcake chocolate candy peppermint candle snowglobe angel poinsettia wrapping paper bow greeting card cookie milk wish dream fireplace kaggle toy doll game night puzzle eggnog as in': 222.93269021691262,
 'decorations eggnog yuletide poinsettia fruitcake scrooge nutcracker mistletoe holly wreath gingerbread cookie snowglobe reindeer angel star merry and the season of joy and wonder and peace to you from the family of the grinch holiday cheer is not as cheer unwrap gifts laugh hohoho sing carol in sleigh drive visit chimney chimney elf naughty nice eat bake sleep dream chocolate peppermint ornament stocking fireplace fireplace advent candle wish hope give card ornament wrapping paper toy doll bow game night night puzzle candy walk jingle jump relax believe it with we have that kaggle workshop workshop polar beard milk greeting magi': 120.92163009638384}
print(len(past))

6


In [7]:
%%time

def get_best_plex(df, past):
    l = 2
    df['score'] = df['text'].map(lambda x: scorer.get_perplexity(x))
    print(np.mean(df['score'].values))
    for r in range(len(df)):
        #get best past
        print("row: ", r, df.at[r, 'score'])
        t = df['text'][r].split(' ')
        for k in past:
            kp = k.split(' ')
            if len(kp)==len(t):
                if sorted(kp) == sorted(t):
                    if df['score'][r] > past[k]:
                        df.at[r, 'score'] = past[k]
                        df.at[r, 'text'] = k
        print("row: ", r, df.at[r, 'score'])
        #for i in tqdm(range(l)):
        for i in range(l):
            t = df['text'][r].split(' ')
            random.shuffle(t)
            t = ' '.join(t)
            if t not in past:
                s = scorer.get_perplexity(t)
                past[t]=s
                if s < df['score'][r]:
                    df.at[r, 'score'] = s
                    df.at[r, 'text'] = t
                    print(r, i, "New Score: ", s, np.mean(df['score'].values))
        with open('past0.pickle', 'wb') as f:
            pickle.dump(past, f, protocol=pickle.HIGHEST_PROTOCOL)
    print('MEAN SCORE: ', np.mean(df['score'].values))
    return df[['id','text']], past

df, past = get_best_plex(df, past)
df.to_csv("submission.csv", index=False)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


1018.7141148854306
row:  0 1350.9216564227258
row:  0 495.6812574407127
row:  1 2146.111641264015
row:  1 514.3236249412977
row:  2 872.250522619955
row:  2 327.18895045897995
row:  3 834.3357780659894
row:  3 327.33220436011106
row:  4 489.6441270405202
row:  4 222.93269021691262
row:  5 419.02096389937867
row:  5 120.92163009638384
MEAN SCORE:  334.73005958573293
CPU times: user 2min 12s, sys: 2.65 s, total: 2min 15s
Wall time: 1min 9s
